In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor()

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
df_pharmacy = pd.read_sql('select frequency, pharmacy_id from pharmacy', con = mimicconn) 
df_pharmacy

In [ ]:
df_pharmacy.dtypes

In [ ]:
df_pharmacy[df_pharmacy['pharmacy_id']== 31327446]

In [ ]:
df_pharmacy['pharmacy_id'].nunique()

In [ ]:
df_pharmacy['frequency'] = df_pharmacy['frequency'].str.lower()

In [ ]:
dict_pharmacyid = dict(zip(df_pharmacy.pharmacy_id, df_pharmacy.frequency))
dict_pharmacyid

In [ ]:
df_drug = pd.read_sql('select drug_id, concept_id, name, dosage_form, route, dose_limit_units from drug ', con = conn)
df_drug

In [ ]:
drugid_dosageform = dict(zip(df_drug.concept_id, df_drug.dosage_form))
len(drugid_dosageform)

In [ ]:
drugid_route = dict(zip(df_drug.concept_id, df_drug.route))
len(drugid_route)

In [ ]:
drugid_units = dict(zip(df_drug.concept_id, df_drug.dose_limit_units))
len(drugid_units)

In [ ]:
drugid_conceptid = dict(zip(df_drug.concept_id, df_drug.drug_id))
len(drugid_conceptid)

In [ ]:
df_order = pd.read_sql('select order_id, concept_id, order_number, instructions from orders ', con = conn)
df_order['order_number'] = df_order['order_number'].astype(int)

In [ ]:
# mapping frequency to order_number

In [ ]:
df_order['frequency'] = df_order['order_number'].map(dict_pharmacyid)

In [ ]:
df_order

In [ ]:
freq_concept = pd.read_sql('select * from concept where class_id = 16 ', con = conn)
freq_concept

In [ ]:
freq_con_dict = dict(zip(freq_concept.description, freq_concept.concept_id))
freq_con_dict

In [ ]:
df_order['frequency'] = df_order['frequency'].map(freq_con_dict)
df_order

In [ ]:
# getting drug_inventory_id

In [ ]:
df_order['drug_inventory_id'] = df_order['concept_id'].map(drugid_conceptid)

In [ ]:
# getting dosing_type

In [ ]:
df_order['dosing_type'] = df_order['concept_id'].map(drugid_dosageform)

In [ ]:
df_order

In [ ]:
dosage_concept = pd.read_sql('select * from concept where class_id = 9 ', con = conn)
dosage_concept['concept_id'] = dosage_concept['concept_id'].astype(float)
dosage_concept_dict = dict(zip(dosage_concept.concept_id, dosage_concept.description))
dosage_concept_dict

In [ ]:
dosage_concept

In [ ]:
df_order['dosing_type'] = df_order['dosing_type'].map(dosage_concept_dict)
df_order = df_order.rename(columns = {'instructions':'dosing_instructions'})


In [ ]:
df_order['route'] = df_order['concept_id'].map(drugid_route)
df_order

In [ ]:
df_order['dose_units'] = df_order['concept_id'].map(drugid_units)
df_order

In [ ]:
df_order.isnull().sum()

In [ ]:
df_order= df_order.replace({np.NAN: None})
df_order

In [ ]:
column_names = ['order_id', 'drug_inventory_id', 'dosing_type', 'dosing_instructions', 'route', 'dose_units', 'frequency' ]
df_order = df_order.reindex(columns = column_names)
df_order

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_order.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_order.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `drug_order` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 
